
#### purpose

Extract the time coverage of data in the cloudnet database



#### useful links

- https://docs.cloudnet.fmi.fi/api/data-portal.html
- https://cloudnet.fmi.fi/api/sites
- https://cloudnet.fmi.fi/api/raw-files?site=norunda&updatedAtFrom=2021-09-01
- https://cloudnet.fmi.fi/api/files?site=norunda&dateFrom=2019-03-01&product=classification


In [1]:
import datetime
# import copy
# import scipy.stats
import numpy as np
# import cartopy
# import cartopy.crs as ccrs
# import matplotlib
# import matplotlib.pyplot as plt
# import toml

import requests
import pprint

from IPython.display import JSON

In [2]:
%matplotlib inline

In [3]:
def dt_to_ts(dt):
    """convert a datetime to unix timestamp"""
    return (dt - datetime.datetime(1970, 1, 1)).total_seconds()


In [4]:


url = 'https://cloudnet.fmi.fi/api/files'
payload = {
    'date': '2020-10-01',
    'product': 'classification'
}
metadata = requests.get(url, payload).json()
for row in metadata:
    print(row)
    #res = requests.get(row['downloadUrl'])
    #with open(row['filename'], 'wb') as f:
    #    f.write(res.content)


{'uuid': '9922dca9-87aa-425d-b705-e1f65eda57b7', 'version': '', 'pid': 'https://hdl.handle.net/21.12132/1.9922dca987aa425d', 'dvasId': '334844', 'volatile': True, 'tombstoneReason': None, 'legacy': False, 'measurementDate': '2020-10-01', 'checksum': 'cd5d4b241ae39d80a618da243c339a77baa2f8eda69b851ea037446a6fcc357b', 'size': '137963', 'format': 'HDF5 (NetCDF4)', 'errorLevel': 'info', 'createdAt': '2021-03-18T16:25:42.388Z', 'updatedAt': '2024-10-04T05:58:59.455Z', 'dvasUpdatedAt': '2024-10-04T05:59:02.606Z', 'startTime': '2020-10-01T07:48:45.000Z', 'stopTime': '2020-10-01T23:59:44.000Z', 'instrumentPid': None, 'instrumentId': None, 'instrumentInfoUuid': None, 'site': {'id': 'lindenberg', 'humanReadableName': 'Lindenberg', 'description': None, 'type': ['cloudnet'], 'latitude': 52.208, 'longitude': 14.118, 'altitude': 104, 'gaw': 'LIN', 'dvasId': '6zm2', 'actrisId': 49, 'country': 'Germany', 'countryCode': 'DE', 'countrySubdivisionCode': None}, 'product': {'id': 'classification', 'humanRe

In [8]:
url = 'https://cloudnet.fmi.fi/api/sites'

sites = requests.get(url).json()
print(len(sites))

pprint.pprint(sites[:2])

for s in sites:
    if s['latitude'] is not None:
        print(f" {s['id']:20s} {s['humanReadableName']:25s} {s['latitude']:8.3f} {s['longitude']:8.3f} {s['altitude']:6.1f} {s['type']}")
    else:
        print(f" {s['id']:20s} {s['humanReadableName']:25s} {s['latitude']} {s['longitude']} {s['altitude']} {s['type']}")
# JSON(sites)

160
[{'actrisId': None,
  'altitude': 167,
  'country': 'United Arab Emirates',
  'countryCode': 'AE',
  'countrySubdivisionCode': None,
  'description': None,
  'dvasId': None,
  'gaw': None,
  'humanReadableName': 'Al Dhaid',
  'id': 'al-dhaid',
  'latitude': 25.234,
  'longitude': 55.976,
  'status': 'inactive',
  'type': ['hidden']},
 {'actrisId': None,
  'altitude': 380,
  'country': 'Norway',
  'countryCode': 'NO',
  'countrySubdivisionCode': None,
  'description': None,
  'dvasId': None,
  'gaw': None,
  'humanReadableName': 'Alomar',
  'id': 'alomar',
  'latitude': 62.278,
  'longitude': 16.008,
  'status': 'inactive',
  'type': ['hidden']}]
 al-dhaid             Al Dhaid                    25.234   55.976  167.0 ['hidden']
 alomar               Alomar                      62.278   16.008  380.0 ['hidden']
 amapola              Amapola                     37.911   -3.228  370.0 ['hidden']
 andenes              Andenes                     69.278   16.009  380.0 ['hidden', 'model

In [13]:
sites_to_scrape = [s for s in sites if not 'hidden' in s['type']]
sites_to_scrape = [s for s in sites]

print(len(sites_to_scrape))
for s in sites_to_scrape:
    #print(f" {s['id']:20s} {s['humanReadableName']:25s} {s['latitude']:8.3f} {s['longitude']:8.3f} {s['altitude']:6.1f} {s['type']}")
    if s['latitude'] is not None:
        print(f" {s['id']:20s} {s['humanReadableName']:25s} {s['latitude']:8.3f} {s['longitude']:8.3f} {s['altitude']:6.1f} {s['type']}")
    else:
        print(f" {s['id']:20s} {s['humanReadableName']:25s} {s['latitude']} {s['longitude']} {s['altitude']} {s['type']}")

160
 al-dhaid             Al Dhaid                    25.234   55.976  167.0 ['hidden']
 alomar               Alomar                      62.278   16.008  380.0 ['hidden']
 amapola              Amapola                     37.911   -3.228  370.0 ['hidden']
 andenes              Andenes                     69.278   16.009  380.0 ['hidden', 'model']
 antikythera          Antikythera                 35.860   23.310  193.0 ['hidden', 'model']
 apukka               Apukka                      66.579   26.011  106.0 ['hidden']
 are                  Åre                         63.402   13.076  400.0 ['hidden']
 arm-andoya           Andøya                      69.141   15.684    2.0 ['arm']
 arm-ascension        Ascension Island            -7.967  -14.350  341.0 ['arm']
 arm-aware            West Antarctic             -77.850  166.730   76.0 ['arm']
 arm-banizoumbou      Banizoumbou                 13.522    2.632  228.0 ['hidden', 'arm']
 arm-bjornoya         Bjørnøya                    74.504

In [21]:
# check for classification file coverage


#url = 'https://cloudnet.fmi.fi/api/files?site=norunda&dateFrom=2019-03-01&product=classification'

url = 'https://cloudnet.fmi.fi/api/files'


for s in sites_to_scrape:
    
    payload = {
        'dateFrom': '1990-01-01',
        'site': s['id'],
        'showLegacy': True,
        'product': 'classification'
    }
    
    files = requests.get(url, payload).json()

    print('===================================================================')
    print(s['id'], ' no files ', len(files))
    #pprint.pprint(files[:2])
    
    if len(files) == 0:
        continue

    dates_classification = sorted([f['measurementDate'] for f in files])
    print(dates_classification[:5], '\n', dates_classification[-5:])
    beginend = min(dates_classification), max(dates_classification)

    ts_classification = [dt_to_ts(datetime.datetime.strptime(d, "%Y-%m-%d")) for d in dates_classification]
    total_duration = (ts_classification[-1] - ts_classification[0] + 3600*24)/(3600*24)
    print('total period ', beginend, ' -> ', total_duration, 'days coverage ', len(files)/total_duration)

    ts_diff = np.diff(ts_classification)
    i_gap = np.where(ts_diff > 14*24*3600)[0]

    long_gaps = []
    print('gaps larger 2 weeks                 duration [days]')
    for i in i_gap:
        interruption_beginend = dates_classification[i], dates_classification[i+1]
        gap_duration = (ts_classification[i+1] - ts_classification[i] - 3600*24)/(3600*24)
        long_gaps.append([interruption_beginend, gap_duration])
        print('   ', interruption_beginend, ' -> ', gap_duration)
    print('duration long gaps', sum([e[1] for e in long_gaps]))

    valid_periods = []
    start = beginend[0]
    for i in i_gap:
        end = dates_classification[i]
        valid_periods.append([start, end])
        start = dates_classification[i+1]
    valid_periods.append([start, beginend[1]])
    print('valid_periods', valid_periods)

al-dhaid  no files  0
alomar  no files  0
amapola  no files  0
andenes  no files  0
antikythera  no files  0
apukka  no files  0
are  no files  0
arm-andoya  no files  112
['2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05'] 
 ['2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21']
total period  ('2019-12-01', '2020-03-21')  ->  112.0 days coverage  1.0
gaps larger 2 weeks                 duration [days]
duration long gaps 0
valid_periods [['2019-12-01', '2020-03-21']]
arm-ascension  no files  136
['2016-10-13', '2016-10-14', '2016-10-15', '2016-10-16', '2016-10-17'] 
 ['2017-02-22', '2017-02-23', '2017-02-24', '2017-02-25', '2017-02-27']
total period  ('2016-10-13', '2017-02-27')  ->  138.0 days coverage  0.9855072463768116
gaps larger 2 weeks                 duration [days]
duration long gaps 0
valid_periods [['2016-10-13', '2017-02-27']]
arm-aware  no files  395
['2015-12-01', '2015-12-02', '2015-12-03', '2015-12-04', '2015-12-05'] 
 ['2016-12-25', '

In [12]:
payload = {
    'dateFrom': '1990-01-01',
    'site': 'punta-arenas',
    'showLegacy': True,
    'product': 'classification'
}

files = requests.get(url, payload).json()

print('===================================================================')
print('punta-arenas', ' no files ', len(files))
#pprint.pprint(files[:2])

dates_classification = sorted([f['measurementDate'] for f in files])
print(dates_classification[:10], dates_classification[-10:])
beginend = min(dates_classification), max(dates_classification)

ts_classification = [dt_to_ts(datetime.datetime.strptime(d, "%Y-%m-%d")) for d in dates_classification]
total_duration = (ts_classification[-1] - ts_classification[0] + 3600*24)/(3600*24)
print('total period ', beginend, ' -> ', total_duration, 'days coverage ', len(files)/total_duration)

ts_diff = np.diff(ts_classification)
i_gap = np.where(ts_diff > 1*24*3600)[0]

long_gaps = []
print('gaps longer 1 day                 duration [days]')
for i in i_gap:
    interruption_beginend = dates_classification[i], dates_classification[i+1]
    gap_duration = (ts_classification[i+1] - ts_classification[i] - 3600*24)/(3600*24)
    long_gaps.append([interruption_beginend, gap_duration])
    print('   ', interruption_beginend, ' -> ', gap_duration)
print('duration long gaps', sum([e[1] for e in long_gaps]))

punta-arenas  no files  1032
['2018-11-27', '2018-11-28', '2018-11-29', '2018-11-30', '2018-12-01', '2018-12-02', '2018-12-03', '2018-12-04', '2018-12-05', '2018-12-06'] ['2021-11-21', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-11-29', '2021-11-30']
total period  ('2018-11-27', '2021-11-30')  ->  1100.0 days coverage  0.9381818181818182
gaps longer 1 day                 duration [days]
    ('2019-11-05', '2019-12-06')  ->  30.0
    ('2019-12-26', '2019-12-28')  ->  1.0
    ('2020-05-02', '2020-05-04')  ->  1.0
    ('2020-06-27', '2020-06-29')  ->  1.0
    ('2020-09-24', '2020-09-30')  ->  5.0
    ('2020-11-24', '2020-12-05')  ->  10.0
    ('2020-12-07', '2020-12-14')  ->  6.0
    ('2021-06-29', '2021-07-02')  ->  2.0
    ('2021-07-31', '2021-08-04')  ->  3.0
    ('2021-08-06', '2021-08-11')  ->  4.0
    ('2021-08-24', '2021-08-27')  ->  2.0
    ('2021-10-15', '2021-10-19')  ->  3.0
duration long gaps 68.0
